In [251]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import talib  
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'SPY']#FAANG portfolio with SPY for market proxy

In [252]:
class DataPreprocessor:
    def __init__(self, tickers,):
        self.tickers = tickers
        self.data = None
        
    def run_pipeline(self, start_date='2015-01-01', end_date='2024-12-31'):
        self._download_data(start_date, end_date)
        self._clean_data()
        self._validate_data()
        return self.data
    
    def _download_data(self, start_date, end_date):
        all_data = []
        
        for ticker in self.tickers:
            
            # Download with adjusted close prices and make column names lower
            df = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
            
            if isinstance(df.columns, pd.MultiIndex): 
                df.columns = [col[0].lower() for col in df.columns]
            else:
                df.columns = [col.lower() for col in df.columns]
            
            df.index.names = [name.lower() if name else 'date' for name in df.index.names]
            
            df = df[['open', 'high', 'low', 'close', 'volume']] # Keep only essential columns
            
            df['ticker'] = ticker # Add ticker column and set multi-index, organize data
            df = df.reset_index()
            df.set_index(['ticker', 'date'], inplace=True)
            
            all_data.append(df)
        
        self.data = pd.concat(all_data, axis=0).sort_index()

    def _clean_data(self):
        if self.data is None:
            raise ValueError("No data to clean!")
        
        # 1. Fill small gaps (forward fill then backward fill)
        self.data = self.data.groupby(level=0, group_keys=False).apply(lambda x: x.ffill().bfill())
        
        # 2. Drop any remaining NaN (usually at the beginning)
        self.data = self.data.dropna()

    def _validate_data(self):
        # Check date alignment
        date_counts = {}
        for ticker in self.data.index.get_level_values(0).unique():
            dates = self.data.xs(ticker, level=0).index
            date_counts[ticker] = len(dates)
        
        if len(set(date_counts.values())) == 1:
            print(f"All tickers have {list(date_counts.values())[0]} trading days")
        else:
            print("tickers have different dates")
        
# Create an instance and run the pipeline
preprocessor = DataPreprocessor(tickers)
data = preprocessor.run_pipeline(start_date='2015-01-01', end_date='2024-12-31')

# Preview the data
print(data.shape)
print(data.index.get_level_values('ticker').unique().tolist())
print(data.index.get_level_values('date').min(), "to", data.index.get_level_values('date').max())
data.groupby(level='ticker').head(3)

All tickers have 2515 trading days
(15090, 5)
['AAPL', 'AMZN', 'GOOGL', 'META', 'NFLX', 'SPY']
2015-01-02 00:00:00 to 2024-12-30 00:00:00


open        high         low       close     volume
ticker date                                                                 
AAPL   2015-01-02   24.694237   24.705322   23.798602   24.237553  212818400
       2015-01-05   24.006988   24.086797   23.368517   23.554737  257142000
       2015-01-06   23.619031   23.816336   23.195599   23.556957  263188400
AMZN   2015-01-02   15.629000   15.737500   15.348000   15.426000   55664000
       2015-01-05   15.350500   15.419000   15.042500   15.109500   55484000
       2015-01-06   15.112000   15.150000   14.619000   14.764500   70380000
GOOGL  2015-01-02   26.430299   26.589101   26.196068   26.278944   26480000
       2015-01-05   26.159842   26.201527   25.693367   25.778225   41182000
       2015-01-06   25.829837   25.865070   25.087943   25.142035   54456000
META   2015-01-02   78.034895   78.382466   77.160995   77.905792   18177500
       2015-01-05   77.439062   78.700249   76.326828   76.654541   26452200
       2015-01-06   76.694267   77.051763   74.837237   75.621758   27399300
NFLX   2015-01-02    4.915143    5.033143    4.873143    4.984857  134750000
       2015-01-05    4.925857    4.925857    4.714714    4.731143  181650000
       2015-01-06    4.734714    4.764000    4.566143    4.650143  160377000
SPY    2015-01-02  171.378492  171.793694  169.551596  170.589600  121465900
       2015-01-05  169.543303  169.709381  167.201575  167.508820  169632600
       2015-01-06  167.816066  168.339223  165.133869  165.931061  209151400

In [253]:
class FeatureEngineer:
    def __init__(self, data):
        self.data = data.copy()
        self.finished_features = []
        
    def run_pipeline(self):
        self.core_price_features()
        self.math_rule_features()
        self.momentum_features()
        self.volatility_features()
        self.volume_features()
        self.lagged_features()
        self.target_variable()
        return self.data
    
    def _apply_by_ticker(self, func): #splits data by ticker for feature calculation and reapplies
        return self.data.groupby(level='ticker', group_keys=False).apply(func)
    
    #create core price features for both models
    def core_price_features(self):
        def calc(df):
            close, high, low, open_ = df['close'], df['high'], df['low'], df['open']
            
            # Returns
            df['log_return'] = np.log(close / close.shift(1))
            df['overnight_return'] = np.log(open_ / close.shift(1))
            df['intraday_return'] = np.log(close / open_)
            # Volatility
            df['volatility_20d'] = df['log_return'].rolling(20).std() * np.sqrt(252)
            df['atr_14'] = talib.ATR(high, low, close, timeperiod=14)
            # SMAs
            sma10 = talib.SMA(close, timeperiod=10)
            sma20 = talib.SMA(close, timeperiod=20)
            sma50 = talib.SMA(close, timeperiod=50)
            #ratios
            df['price_sma20_ratio'] = close / sma20
            df['price_sma50_ratio'] = close / sma50
            df['sma10_sma20_ratio'] = sma10 / sma20
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['log_return', 'overnight_return', 'intraday_return', 
                            'volatility_20d', 'atr_14', 'price_sma20_ratio', 
                            'price_sma50_ratio', 'sma10_sma20_ratio']
    
    # Rule-Based Binary Features 
    
    def math_rule_features(self):
        """15 binary features for rule-based model"""
        def calc(df):
            close, high, low, volume = df['close'], df['high'], df['low'], df['volume']
            
            # SMAs
            sma10 = talib.SMA(close, timeperiod=10)
            sma20 = talib.SMA(close, timeperiod=20)
            sma50 = talib.SMA(close, timeperiod=50)
            sma200 = talib.SMA(close, timeperiod=200)
            
            # Trends
            df['golden_cross'] = (sma50 > sma200).astype(int)
            df['short_uptrend'] = (sma10 > sma20).astype(int)
            df['price_above_sma20'] = (close > sma20).astype(int)
            df['price_above_sma50'] = (close > sma50).astype(int)
            
            # Momentum
            rsi = talib.RSI(close, timeperiod=14)
            macd, signal, _ = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
            stoch_k, _ = talib.STOCH(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3)
            roc = talib.ROC(close, timeperiod=10)
            
            df['rsi_oversold'] = (rsi < 30).astype(int)
            df['rsi_overbought'] = (rsi > 70).astype(int)
            df['macd_bullish'] = (macd > signal).astype(int)
            df['roc_positive'] = (roc > 0).astype(int)
            df['stoch_oversold'] = (stoch_k < 20).astype(int)
            
            # Volatility/Reversion 
            upper, _, lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2)
            bb_pos = (close - lower) / (upper - lower)
            vol_20d = df['log_return'].rolling(20).std() * np.sqrt(252)
            vol_75pct = vol_20d.expanding().quantile(0.75)
            
            df['bb_oversold'] = (bb_pos < 0.2).astype(int)
            df['bb_overbought'] = (bb_pos > 0.8).astype(int)
            df['high_volatility'] = (vol_20d > vol_75pct).astype(int)
            
            # Volume 
            vol_sma20 = talib.SMA(volume, timeperiod=20)
            vol_ratio = volume / vol_sma20
            price_up = close > close.shift(1)
            price_down = close < close.shift(1)
            
            df['volume_spike'] = (vol_ratio > 1.5).astype(int)
            df['volume_confirmation'] = (price_up & (vol_ratio > 1)).astype(int)
            df['volume_divergence'] = (price_down & (vol_ratio > 1)).astype(int)
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['golden_cross', 'short_uptrend', 'price_above_sma20', 'price_above_sma50',
                            'rsi_oversold', 'rsi_overbought', 'macd_bullish', 'roc_positive', 'stoch_oversold',
                            'bb_oversold', 'bb_overbought', 'high_volatility',
                            'volume_spike', 'volume_confirmation', 'volume_divergence']
    
    # XGBoost features
    
    def momentum_features(self):
        """5 momentum oscillators"""
        def calc(df):
            close, high, low = df['close'], df['high'], df['low']
            
            df['rsi_14'] = talib.RSI(close, timeperiod=14)
            macd, signal, hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
            df['macd_histogram'] = hist
            df['stoch_k'], _ = talib.STOCH(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3)
            df['williams_r'] = talib.WILLR(high, low, close, timeperiod=14)
            df['roc_10'] = talib.ROC(close, timeperiod=10)
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['rsi_14', 'macd_histogram', 'stoch_k', 'williams_r', 'roc_10']
    
    def volatility_features(self):
        """4 volatility & range features"""
        def calc(df):
            close, high, low = df['close'], df['high'], df['low']
            
            # Bollinger Bands
            upper, middle, lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2)
            df['bb_position'] = (close - lower) / (upper - lower)
            df['bb_width'] = (upper - lower) / middle
            
            # Parkinson volatility (high-low based)
            df['parkinson_vol'] = np.sqrt((1 / (4 * np.log(2))) * (np.log(high / low) ** 2)).rolling(20).mean()
            
            # Short-term volatility
            df['returns_std_5d'] = df['log_return'].rolling(5).std()
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['bb_position', 'bb_width', 'parkinson_vol', 'returns_std_5d']
    
    def volume_features(self):
        """3 volume indicators"""
        def calc(df):
            close, volume = df['close'], df['volume']
            
            vol_sma20 = talib.SMA(volume, timeperiod=20)
            df['volume_ratio'] = volume / vol_sma20
            df['obv'] = talib.OBV(close, volume)
            df['volume_zscore'] = (volume - volume.rolling(20).mean()) / volume.rolling(20).std()
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['volume_ratio', 'obv', 'volume_zscore']
    
    
    # Derived and time features 

    def lagged_features(self):
        """6 lagged features (1-day lag to avoid look-ahead bias)"""
        lag_cols = ['log_return', 'rsi_14', 'volume_ratio', 'macd_histogram', 'bb_position', 'atr_14']
        
        def calc(df):
            for col in lag_cols:
                df[f'{col}_lag1'] = df[col].shift(1)
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += [f'{col}_lag1' for col in lag_cols]
    
    def target_variable(self):
        """Target: next day's return (shifted to avoid look-ahead bias)"""
        def calc(df):
            df['target'] = df['log_return'].shift(-1)  # Predict tomorrow's return
            return df
        self.data = self._apply_by_ticker(calc)

# Run the feature engineering pipeline
fe = FeatureEngineer(data)
features_data = fe.run_pipeline()

# Preview

features_data.groupby(level='ticker').head(3)

open        high         low       close     volume  \
ticker date                                                                    
AAPL   2015-01-02   24.694237   24.705322   23.798602   24.237553  212818400   
       2015-01-05   24.006988   24.086797   23.368517   23.554737  257142000   
       2015-01-06   23.619031   23.816336   23.195599   23.556957  263188400   
AMZN   2015-01-02   15.629000   15.737500   15.348000   15.426000   55664000   
       2015-01-05   15.350500   15.419000   15.042500   15.109500   55484000   
       2015-01-06   15.112000   15.150000   14.619000   14.764500   70380000   
GOOGL  2015-01-02   26.430299   26.589101   26.196068   26.278944   26480000   
       2015-01-05   26.159842   26.201527   25.693367   25.778225   41182000   
       2015-01-06   25.829837   25.865070   25.087943   25.142035   54456000   
META   2015-01-02   78.034895   78.382466   77.160995   77.905792   18177500   
       2015-01-05   77.439062   78.700249   76.326828   76.654541   26452200   
       2015-01-06   76.694267   77.051763   74.837237   75.621758   27399300   
NFLX   2015-01-02    4.915143    5.033143    4.873143    4.984857  134750000   
       2015-01-05    4.925857    4.925857    4.714714    4.731143  181650000   
       2015-01-06    4.734714    4.764000    4.566143    4.650143  160377000   
SPY    2015-01-02  171.378492  171.793694  169.551596  170.589600  121465900   
       2015-01-05  169.543303  169.709381  167.201575  167.508820  169632600   
       2015-01-06  167.816066  168.339223  165.133869  165.931061  209151400   

                   log_return  overnight_return  intraday_return  \
ticker date                                                        
AAPL   2015-01-02         NaN               NaN        -0.018667   
       2015-01-05   -0.028576         -0.009558        -0.019018   
       2015-01-06    0.000094          0.002726        -0.002632   
AMZN   2015-01-02         NaN               NaN        -0.013074   
       2015-01-05   -0.020731         -0.004906        -0.015824   
       2015-01-06   -0.023098          0.000165        -0.023264   
GOOGL  2015-01-02         NaN               NaN        -0.005743   
       2015-01-05   -0.019238         -0.004543        -0.014695   
       2015-01-06   -0.024989          0.002000        -0.026989   
META   2015-01-02         NaN               NaN        -0.001656   
       2015-01-05   -0.016191         -0.006009        -0.010182   
       2015-01-06   -0.013565          0.000518        -0.014083   
NFLX   2015-01-02         NaN               NaN         0.014084   
       2015-01-05   -0.052238         -0.011906        -0.040331   
       2015-01-06   -0.017269          0.000755        -0.018023   
SPY    2015-01-02         NaN               NaN        -0.004614   
       2015-01-05   -0.018225         -0.006152        -0.012072   
       2015-01-06   -0.009464          0.001833        -0.011296   

                   volatility_20d  atr_14  ...  volume_ratio          obv  \
ticker date                                ...                              
AAPL   2015-01-02             NaN     NaN  ...           NaN  212818400.0   
       2015-01-05             NaN     NaN  ...           NaN  -44323600.0   
       2015-01-06             NaN     NaN  ...           NaN  218864800.0   
AMZN   2015-01-02             NaN     NaN  ...           NaN   55664000.0   
       2015-01-05             NaN     NaN  ...           NaN     180000.0   
       2015-01-06             NaN     NaN  ...           NaN  -70200000.0   
GOOGL  2015-01-02             NaN     NaN  ...           NaN   26480000.0   
       2015-01-05             NaN     NaN  ...           NaN  -14702000.0   
       2015-01-06             NaN     NaN  ...           NaN  -69158000.0   
META   2015-01-02             NaN     NaN  ...           NaN   18177500.0   
       2015-01-05             NaN     NaN  ...           NaN   -8274700.0   
       2015-01-06             NaN     NaN  ...           NaN  

In [254]:
class RuleBasedModel:
    def __init__(self, data):
        self.data = data.copy()
        self.binary_features = [
            'golden_cross', 'short_uptrend', 'price_above_sma20', 'price_above_sma50',
            'rsi_oversold', 'rsi_overbought', 'macd_bullish', 'roc_positive', 'stoch_oversold',
            'bb_oversold', 'bb_overbought', 'high_volatility',
            'volume_spike', 'volume_confirmation', 'volume_divergence'
        ]
        # Default weights: positive = bullish signal, negative = bearish signal
        self.weights = {
            # Trend (bullish)
            'golden_cross': 2.0,
            'short_uptrend': 1.5,
            'price_above_sma20': 1.0,
            'price_above_sma50': 1.0,
            # Momentum
            'rsi_oversold': 1.5,        # Oversold = buy opportunity
            'rsi_overbought': -1.5,     # Overbought = sell signal
            'macd_bullish': 1.5,
            'roc_positive': 1.0,
            'stoch_oversold': 1.0,
            # Volatility/Reversion
            'bb_oversold': 1.5,         # Mean reversion buy
            'bb_overbought': -1.5,      # Mean reversion sell
            'high_volatility': -0.5,    # High vol = reduce exposure
            # Volume
            'volume_spike': 0.5,
            'volume_confirmation': 1.5, # Price up + volume = strong
            'volume_divergence': -1.5   # Price down + volume = weak
        }
        self.results = None
    
    def run_pipeline(self):
        self._calculate_scores()
        self._generate_signals()
        self._evaluate_performance()
    
    def _apply_by_ticker(self, func):
        return self.data.groupby(level='ticker', group_keys=False).apply(func)
    
    def _calculate_scores(self):
        """Calculate weighted score from binary features"""
        self.data['rule_score'] = sum(
            self.data[feat] * self.weights[feat] for feat in self.binary_features
        )
        # Normalize to [-1, 1] range
        max_pos = sum(w for w in self.weights.values() if w > 0)
        max_neg = abs(sum(w for w in self.weights.values() if w < 0))
        self.data['rule_score_norm'] = self.data['rule_score'].apply(
            lambda x: x / max_pos if x > 0 else x / max_neg if x < 0 else 0
        )
    
    def _generate_signals(self):
        """Convert scores to trading signals: 1=long, 0=neutral, -1=short"""
        # Thresholds for signal generation
        long_threshold = 0.2
        short_threshold = -0.2
        
        self.data['signal'] = 0
        self.data.loc[self.data['rule_score_norm'] > long_threshold, 'signal'] = 1
        self.data.loc[self.data['rule_score_norm'] < short_threshold, 'signal'] = -1
        
        # Strategy return: today's signal * tomorrow's return (shift signal forward)
        # Signal is made EOD, executed next day, so we shift the signal
        self.data['strategy_return'] = self.data['signal'].shift(1) * self.data['log_return']
    
    def _evaluate_performance(self):
        """Calculate performance metrics per ticker and overall"""
        def calc_metrics(df):
            df = df.dropna(subset=['strategy_return', 'log_return'])
            if len(df) == 0:
                return pd.Series()
            
            strat_ret = df['strategy_return']
            buy_hold = df['log_return']  # Use actual returns, not shifted target
            
            # Annualized metrics
            trading_days = 252
            
            # Cumulative returns
            strat_cum = (1 + strat_ret).prod() - 1
            bh_cum = (1 + buy_hold).prod() - 1
            
            # Sharpe ratio (annualized)
            sharpe = (strat_ret.mean() / strat_ret.std()) * np.sqrt(trading_days) if strat_ret.std() > 0 else 0
            
            # Max drawdown
            cum_returns = (1 + strat_ret).cumprod()
            rolling_max = cum_returns.expanding().max()
            drawdown = (cum_returns - rolling_max) / rolling_max
            max_dd = drawdown.min()
            
            # Win rate
            wins = (strat_ret > 0).sum()
            total_trades = (df['signal'] != 0).sum()
            win_rate = wins / total_trades if total_trades > 0 else 0
            
            return pd.Series({
                'total_return': strat_cum,
                'buy_hold_return': bh_cum,
                'sharpe_ratio': sharpe,
                'max_drawdown': max_dd,
                'win_rate': win_rate,
                'n_trades': total_trades,
                'n_days': len(df)
            })
        
        # Per-ticker results
        self.results = self.data.groupby(level='ticker').apply(calc_metrics)
        
        # Print summary
        print("=" * 60)
        print("RULE-BASED MODEL PERFORMANCE")
        print("=" * 60)
        for ticker in self.results.index:
            r = self.results.loc[ticker]
            print(f"\n{ticker}:")
            print(f"  Strategy Return: {r['total_return']*100:>8.2f}%  |  Buy & Hold: {r['buy_hold_return']*100:>8.2f}%")
            print(f"  Sharpe Ratio:    {r['sharpe_ratio']:>8.2f}   |  Max Drawdown: {r['max_drawdown']*100:>7.2f}%")
            print(f"  Win Rate:        {r['win_rate']*100:>8.1f}%  |  Trades: {int(r['n_trades'])}")
        
        # Overall
        overall = self.data.dropna(subset=['strategy_return'])
        overall_sharpe = (overall['strategy_return'].mean() / overall['strategy_return'].std()) * np.sqrt(252)
        print(f"\n{'='*60}")
        print(f"OVERALL SHARPE: {overall_sharpe:.3f}")

# Run the rule-based model
rb_model = RuleBasedModel(features_data)
rb_model.run_pipeline()

RULE-BASED MODEL PERFORMANCE

AAPL:
  Strategy Return:   495.37%  |  Buy & Hold:   590.19%
  Sharpe Ratio:        0.87   |  Max Drawdown:  -39.86%
  Win Rate:            53.8%  |  Trades: 2011

AMZN:
  Strategy Return:  1047.47%  |  Buy & Hold:   742.62%
  Sharpe Ratio:        1.03   |  Max Drawdown:  -33.09%
  Win Rate:            53.8%  |  Trades: 2091

GOOGL:
  Strategy Return:   254.41%  |  Buy & Hold:   384.22%
  Sharpe Ratio:        0.64   |  Max Drawdown:  -44.23%
  Win Rate:            53.6%  |  Trades: 2053

META:
  Strategy Return:   164.92%  |  Buy & Hold:   262.34%
  Sharpe Ratio:        0.47   |  Max Drawdown:  -73.90%
  Win Rate:            52.2%  |  Trades: 2035

NFLX:
  Strategy Return:   239.46%  |  Buy & Hold:   555.64%
  Sharpe Ratio:        0.52   |  Max Drawdown:  -74.41%
  Win Rate:            50.4%  |  Trades: 2071

SPY:
  Strategy Return:   140.00%  |  Buy & Hold:   191.25%
  Sharpe Ratio:        0.68   |  Max Drawdown:  -26.08%
  Win Rate:            54.5%  |  

In [255]:
# Prepare data
feature_cols = fe.finished_features
model_df = features_data.dropna(subset=feature_cols + ['target']).copy()

# Split by DATE (not row index) to avoid cross-ticker leakage
dates = model_df.index.get_level_values('date').unique().sort_values()
train_end = dates[int(len(dates) * 0.8)]  # 80% train

train_mask = model_df.index.get_level_values('date') <= train_end
test_mask = model_df.index.get_level_values('date') > train_end

P_train = model_df.loc[train_mask, feature_cols]
P_test = model_df.loc[test_mask, feature_cols]
y_train = model_df.loc[train_mask, 'target']  # Use actual returns for regression
y_test = model_df.loc[test_mask, 'target']

# Train XGBoost Regressor (predicts return magnitude, not just direction)
from xgboost import XGBRegressor
model = XGBRegressor(
    n_estimators=300, learning_rate=0.05, max_depth=10,
    subsample=0.7, colsample_bytree=0.8, random_state=42
)
model.fit(P_train, y_train)

# Predict returns and generate signals
test_df = model_df[test_mask].copy()
predictions = model.predict(P_test)
test_df['xgb_pred_return'] = predictions

# Generate signals based on predicted return magnitude
return_threshold = 0.001  # Only trade if predicted |return| > 0.1%
test_df['signal'] = 0
test_df.loc[test_df['xgb_pred_return'] > return_threshold, 'signal'] = 1   # Long
test_df.loc[test_df['xgb_pred_return'] < -return_threshold, 'signal'] = -1  # Short

# Calculate strategy returns: yesterday's signal * today's actual return
# Shift signal to align: signal on day T predicts return on day T+1
test_df['strategy_return'] = test_df['signal'].shift(1) * test_df['log_return']

# Reuse RuleBasedModel evaluation on test data
print("=" * 60)
print("XGBOOST MODEL PERFORMANCE (Test Period)")
print("=" * 60)
xgb_eval = RuleBasedModel(test_df)
xgb_eval.data = test_df
xgb_eval._evaluate_performance()

# Also evaluate Rule-Based on SAME test period for fair comparison
print("\n" + "=" * 60)
print("RULE-BASED MODEL PERFORMANCE (Same Test Period)")
print("=" * 60)
rb_test_data = model_df[model_df.index.get_level_values('date') > train_end]
rb_test = RuleBasedModel(rb_test_data)
rb_test.run_pipeline()

print(f"\nTrain period: {dates[0].date()} to {train_end.date()}")
print(f"Test period:  {dates[int(len(dates)*0.8)+1].date()} to {dates[-1].date()}")

XGBOOST MODEL PERFORMANCE (Test Period)
RULE-BASED MODEL PERFORMANCE

AAPL:
  Strategy Return:   -18.79%  |  Buy & Hold:    83.38%
  Sharpe Ratio:       -0.45   |  Max Drawdown:  -25.26%
  Win Rate:            47.4%  |  Trades: 386

AMZN:
  Strategy Return:    -6.51%  |  Buy & Hold:   114.57%
  Sharpe Ratio:        0.03   |  Max Drawdown:  -35.26%
  Win Rate:            51.2%  |  Trades: 426

GOOGL:
  Strategy Return:    98.24%  |  Buy & Hold:    95.15%
  Sharpe Ratio:        1.47   |  Max Drawdown:  -27.26%
  Win Rate:            54.4%  |  Trades: 401

META:
  Strategy Return:    21.70%  |  Buy & Hold:   286.24%
  Sharpe Ratio:        0.46   |  Max Drawdown:  -38.16%
  Win Rate:            49.7%  |  Trades: 433

NFLX:
  Strategy Return:   -16.13%  |  Buy & Hold:   146.87%
  Sharpe Ratio:       -0.14   |  Max Drawdown:  -56.21%
  Win Rate:            51.9%  |  Trades: 405

SPY:
  Strategy Return:     6.35%  |  Buy & Hold:    51.68%
  Sharpe Ratio:        0.35   |  Max Drawdown:   -8.46

In [256]:
# ==============================================================================
# MODEL COMPARISON ON TEST PERIOD
# ==============================================================================

# 1. Train XGBoost on train data
feature_cols = fe.finished_features
model_df = features_data.dropna(subset=feature_cols + ['target']).copy()

dates = model_df.index.get_level_values('date').unique().sort_values()
train_end = dates[int(len(dates) * 0.8)]

train_mask = model_df.index.get_level_values('date') <= train_end
test_mask = model_df.index.get_level_values('date') > train_end

P_train = model_df.loc[train_mask, feature_cols]
P_test = model_df.loc[test_mask, feature_cols]
y_train = model_df.loc[train_mask, 'target']

# Train XGBoost Regressor
from xgboost import XGBRegressor
xgb_model = XGBRegressor(
    n_estimators=300, learning_rate=0.05, max_depth=4,
    subsample=0.7, colsample_bytree=0.8, random_state=42
)
xgb_model.fit(P_train, y_train)

# 2. Get test data (same for both models)
test_data = model_df[test_mask].copy()

# 3. Generate XGBoost signals
predictions = xgb_model.predict(P_test)
test_data['xgb_pred'] = predictions
test_data['xgb_signal'] = 0
return_threshold = 0.001
test_data.loc[test_data['xgb_pred'] > return_threshold, 'xgb_signal'] = 1
test_data.loc[test_data['xgb_pred'] < -return_threshold, 'xgb_signal'] = -1
test_data['xgb_return'] = test_data['xgb_signal'].shift(1) * test_data['log_return']

# 4. Generate Rule-Based signals on same test data
rb_test = RuleBasedModel(test_data)
rb_test._calculate_scores()
rb_test._generate_signals()
test_data['rb_signal'] = rb_test.data['signal']
test_data['rb_return'] = rb_test.data['strategy_return']

# 5. Calculate metrics for both models
def calculate_metrics(returns_col, signal_col, test_df):
    """Calculate performance metrics"""
    results = {}
    for ticker in test_df.index.get_level_values('ticker').unique():
        ticker_data = test_df.xs(ticker, level='ticker').dropna(subset=[returns_col, 'log_return'])
        
        if len(ticker_data) == 0:
            continue
            
        strat_ret = ticker_data[returns_col]
        buy_hold = ticker_data['log_return']  # Use actual log_return for buy-hold
        signals = ticker_data[signal_col]
        
        # Cumulative returns
        strat_cum = (1 + strat_ret).prod() - 1
        bh_cum = (1 + buy_hold).prod() - 1
        
        # Sharpe ratio
        sharpe = (strat_ret.mean() / strat_ret.std()) * np.sqrt(252) if strat_ret.std() > 0 else 0
        
        # Max drawdown
        cum_rets = (1 + strat_ret).cumprod()
        max_dd = ((cum_rets - cum_rets.expanding().max()) / cum_rets.expanding().max()).min()
        
        # Win rate
        wins = (strat_ret > 0).sum()
        trades = (signals != 0).sum()
        win_rate = wins / trades if trades > 0 else 0
        
        results[ticker] = {
            'return': strat_cum,
            'bh_return': bh_cum,
            'sharpe': sharpe,
            'max_dd': max_dd,
            'win_rate': win_rate,
            'trades': trades
        }
    
    # Overall metrics
    all_returns = test_df.dropna(subset=[returns_col])[returns_col]
    overall_sharpe = (all_returns.mean() / all_returns.std()) * np.sqrt(252) if all_returns.std() > 0 else 0
    
    return results, overall_sharpe

xgb_results, xgb_overall = calculate_metrics('xgb_return', 'xgb_signal', test_data)
rb_results, rb_overall = calculate_metrics('rb_return', 'rb_signal', test_data)

# 6. Print comparison
print("=" * 80)
print("MODEL COMPARISON - TEST PERIOD ONLY")
print("=" * 80)
print(f"Period: {dates[int(len(dates)*0.8)+1].date()} to {dates[-1].date()}")
print("=" * 80)

for ticker in sorted(xgb_results.keys()):
    xgb = xgb_results[ticker]
    rb = rb_results[ticker]
    
    print(f"\n{ticker}:")
    print(f"  {'Metric':<20} {'XGBoost':>12} {'Rule-Based':>12} {'Difference':>12}")
    print(f"  {'-'*20} {'-'*12} {'-'*12} {'-'*12}")
    print(f"  {'Strategy Return':<20} {xgb['return']*100:>11.2f}% {rb['return']*100:>11.2f}% {(xgb['return']-rb['return'])*100:>11.2f}%")
    print(f"  {'Buy & Hold':<20} {xgb['bh_return']*100:>11.2f}% {rb['bh_return']*100:>11.2f}%")
    print(f"  {'Sharpe Ratio':<20} {xgb['sharpe']:>12.2f} {rb['sharpe']:>12.2f} {xgb['sharpe']-rb['sharpe']:>12.2f}")
    print(f"  {'Max Drawdown':<20} {xgb['max_dd']*100:>11.2f}% {rb['max_dd']*100:>11.2f}% {(xgb['max_dd']-rb['max_dd'])*100:>11.2f}%")
    print(f"  {'Win Rate':<20} {xgb['win_rate']*100:>11.1f}% {rb['win_rate']*100:>11.1f}% {(xgb['win_rate']-rb['win_rate'])*100:>11.1f}%")
    print(f"  {'Trades':<20} {xgb['trades']:>12} {rb['trades']:>12} {xgb['trades']-rb['trades']:>12}")

print(f"\n{'='*80}")
print(f"OVERALL SHARPE RATIO:")
print(f"  XGBoost:    {xgb_overall:>6.3f}")
print(f"  Rule-Based: {rb_overall:>6.3f}")
print(f"  Difference: {xgb_overall - rb_overall:>6.3f}")
print("=" * 80)

MODEL COMPARISON - TEST PERIOD ONLY
Period: 2023-01-13 to 2024-12-27

AAPL:
  Metric                    XGBoost   Rule-Based   Difference
  -------------------- ------------ ------------ ------------
  Strategy Return             9.54%       78.81%      -69.28%
  Buy & Hold                 83.38%       83.38%
  Sharpe Ratio                 0.39         1.58        -1.19
  Max Drawdown              -23.25%      -15.68%       -7.58%
  Win Rate                    51.6%        57.1%        -5.5%
  Trades                        182          427         -245

AMZN:
  Metric                    XGBoost   Rule-Based   Difference
  -------------------- ------------ ------------ ------------
  Strategy Return           -13.04%      102.03%     -115.07%
  Buy & Hold                114.57%      114.57%
  Sharpe Ratio                -0.12         1.41        -1.54
  Max Drawdown              -28.05%      -19.02%       -9.03%
  Win Rate                    50.4%        53.3%        -2.9%
  Trades     